In [1]:
import pandas as pd
import json
import re
import nltk
import string
import ast
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
traindf=pd.read_csv(r"../dataset\data\train.tsv", sep='\t', names=["text", "class", "id"])

In [3]:
traindf.shape

(43410, 3)

In [4]:
traindf.head(15)

,text,class,id
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,26,edvnz26
6,Yes I heard abt the f bombs! That has to be wh...,15,ee3b6wu
7,We need more boards and to create a bit more s...,"8,20",ef4qmod
8,Damn youtube and outrage drama is super lucrat...,0,ed8wbdn
9,It might be linked to the trust factor of your...,27,eczgv1o


In [ ]:
validdf=pd.read_csv(r"..\dataset\data\dev.tsv", sep='\t',names=["text", "class", "id"])

In [6]:
validdf.shape

(5426, 3)

In [ ]:
testdf=pd.read_csv(r"..\dataset\data\test.tsv",sep='\t',names=["text","class","id"])

In [8]:
testdf.shape

(5427, 3)

In [9]:
print(traindf.shape[0]+validdf.shape[0]+testdf.shape[0])

54263


In [10]:
df= pd.concat([traindf, validdf, testdf], ignore_index=True)

In [11]:
df.shape

(54263, 3)

In [12]:
df.isna().sum()

text     0
class    0
id       0
dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54263 entries, 0 to 54262
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    54263 non-null  object
 1   class   54263 non-null  object
 2   id      54263 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [14]:
df['class list'] = df['class'].apply(lambda x: str(x).split(','))
df['class length'] = df['class list'].apply(lambda x: len(x))

In [15]:
df.head(15)

,text,class,id,class list,class length
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1
3,To make her feel threatened,14,ed7ypvh,[14],1
4,Dirty Southern Wankers,3,ed0bdzj,[3],1
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,26,edvnz26,[26],1
6,Yes I heard abt the f bombs! That has to be wh...,15,ee3b6wu,[15],1
7,We need more boards and to create a bit more s...,"8,20",ef4qmod,"[8, 20]",2
8,Damn youtube and outrage drama is super lucrat...,0,ed8wbdn,[0],1
9,It might be linked to the trust factor of your...,27,eczgv1o,[27],1


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54263 entries, 0 to 54262
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   text          54263 non-null  object
 1   class         54263 non-null  object
 2   id            54263 non-null  object
 3   class list    54263 non-null  object
 4   class length  54263 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


In [17]:
df["class length"].value_counts()

class length
1    45446
2     8124
3      655
4       37
5        1
Name: count, dtype: int64

In [ ]:
with open(r'..\dataset\data\emotions.txt', 'r') as f:
    emo_list = f.read().splitlines()

In [19]:
emo_list

['admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']

In [20]:
emo_labeled = {emotion: i for i, emotion in enumerate(emo_list)}

In [21]:
emo_labeled

{'admiration': 0,
 'amusement': 1,
 'anger': 2,
 'annoyance': 3,
 'approval': 4,
 'caring': 5,
 'confusion': 6,
 'curiosity': 7,
 'desire': 8,
 'disappointment': 9,
 'disapproval': 10,
 'disgust': 11,
 'embarrassment': 12,
 'excitement': 13,
 'fear': 14,
 'gratitude': 15,
 'grief': 16,
 'joy': 17,
 'love': 18,
 'nervousness': 19,
 'optimism': 20,
 'pride': 21,
 'realization': 22,
 'relief': 23,
 'remorse': 24,
 'sadness': 25,
 'surprise': 26,
 'neutral': 27}

In [ ]:
with open('../dataset/data/ekman_mapping.json') as file:
    ekman_mapping = json.load(file)
ekman_mapping

{'anger': ['anger', 'annoyance', 'disapproval'],
 'disgust': ['disgust'],
 'fear': ['fear', 'nervousness'],
 'joy': ['joy',
  'amusement',
  'approval',
  'excitement',
  'gratitude',
  'love',
  'optimism',
  'relief',
  'pride',
  'admiration',
  'desire',
  'caring'],
 'sadness': ['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse'],
 'surprise': ['surprise', 'realization', 'confusion', 'curiosity']}

In [23]:
def idx2class(idx_list):
    cls = []
    for i in idx_list:
        cls.append(emo_list[int(i)])
    return cls

In [24]:
df['emotion'] = df['class list'].apply(idx2class)

In [25]:
df.head(40)

,text,class,id,class list,class length,emotion
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral]
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral]
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger]
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear]
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance]
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,26,edvnz26,[26],1,[surprise]
6,Yes I heard abt the f bombs! That has to be wh...,15,ee3b6wu,[15],1,[gratitude]
7,We need more boards and to create a bit more s...,"8,20",ef4qmod,"[8, 20]",2,"[desire, optimism]"
8,Damn youtube and outrage drama is super lucrat...,0,ed8wbdn,[0],1,[admiration]
9,It might be linked to the trust factor of your...,27,eczgv1o,[27],1,[neutral]


In [26]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [27]:
negations = {"not", "no", "never", "n't"}
stop_words = stop_words.difference(negations)

In [28]:
def clean(text):
    text = text.lower()
    text = re.sub(r"["
                  "\U0001F600-\U0001F64F"
                  "\U0001F300-\U0001F5FF"
                  "\U0001F680-\U0001F6FF"
                  "\U0001F1E0-\U0001F1FF"
                  "\U00002700-\U000027BF"
                  "\U000024C2-\U0001F251"
                  "]+", '', text)
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"<.*?>", "", text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

In [29]:
df['clean text'] = df['text'].apply(clean)

In [30]:
df.head(50)

,text,class,id,class list,class length,emotion,clean text
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],favourite food anything didnt cook
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],everyone think he laugh screwing people instea...
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],fuck bayless isoing
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],make feel threatened
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],dirty southern wanker
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,26,edvnz26,[26],1,[surprise],omg peyton isnt good enough help u playoff dum...
6,Yes I heard abt the f bombs! That has to be wh...,15,ee3b6wu,[15],1,[gratitude],yes heard abt f bomb thanks reply hubby anxiou...
7,We need more boards and to create a bit more s...,"8,20",ef4qmod,"[8, 20]",2,"[desire, optimism]",need board create bit space name we’ll good
8,Damn youtube and outrage drama is super lucrat...,0,ed8wbdn,[0],1,[admiration],damn youtube outrage drama super lucrative reddit
9,It might be linked to the trust factor of your...,27,eczgv1o,[27],1,[neutral],might linked trust factor friend


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54263 entries, 0 to 54262
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   text          54263 non-null  object
 1   class         54263 non-null  object
 2   id            54263 non-null  object
 3   class list    54263 non-null  object
 4   class length  54263 non-null  int64 
 5   emotion       54263 non-null  object
 6   clean text    54263 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.9+ MB


In [32]:
df['text'] = df['text'].astype('string')
df['id'] = df['id'].astype('string')
df['clean text'] = df['clean text'].astype('string')

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54263 entries, 0 to 54262
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   text          54263 non-null  string
 1   class         54263 non-null  object
 2   id            54263 non-null  string
 3   class list    54263 non-null  object
 4   class length  54263 non-null  int64 
 5   emotion       54263 non-null  object
 6   clean text    54263 non-null  string
dtypes: int64(1), object(3), string(3)
memory usage: 2.9+ MB


In [34]:
def fix_val(x):
    if isinstance(x, str):
        return ast.literal_eval(x)
    else:
        return x

In [35]:
df['class list'] = df['class list'].apply(fix_val)
df['emotion'] = df['emotion'].apply(fix_val)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54263 entries, 0 to 54262
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   text          54263 non-null  string
 1   class         54263 non-null  object
 2   id            54263 non-null  string
 3   class list    54263 non-null  object
 4   class length  54263 non-null  int64 
 5   emotion       54263 non-null  object
 6   clean text    54263 non-null  string
dtypes: int64(1), object(3), string(3)
memory usage: 2.9+ MB


In [37]:
def encode_emotions(emotions):
    vector = [0] * len(emo_labeled)
    for emotion in emotions:
        vector[emo_labeled[emotion]] = 1
    return vector

In [38]:
df['emotion vector'] = df['emotion'].apply(encode_emotions)

In [39]:
df.head(7)

,text,class,id,class list,class length,emotion,clean text,emotion vector
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],favourite food anything didnt cook,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],everyone think he laugh screwing people instea...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],fuck bayless isoing,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],make feel threatened,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],dirty southern wanker,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,26,edvnz26,[26],1,[surprise],omg peyton isnt good enough help u playoff dum...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,Yes I heard abt the f bombs! That has to be wh...,15,ee3b6wu,[15],1,[gratitude],yes heard abt f bomb thanks reply hubby anxiou...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [40]:
df.iloc[0][7]

C:\Users\ree\AppData\Local\Temp\ipykernel_13280\2841602446.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.iloc[0][7]


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1]

In [41]:
emo_labeled

{'admiration': 0,
 'amusement': 1,
 'anger': 2,
 'annoyance': 3,
 'approval': 4,
 'caring': 5,
 'confusion': 6,
 'curiosity': 7,
 'desire': 8,
 'disappointment': 9,
 'disapproval': 10,
 'disgust': 11,
 'embarrassment': 12,
 'excitement': 13,
 'fear': 14,
 'gratitude': 15,
 'grief': 16,
 'joy': 17,
 'love': 18,
 'nervousness': 19,
 'optimism': 20,
 'pride': 21,
 'realization': 22,
 'relief': 23,
 'remorse': 24,
 'sadness': 25,
 'surprise': 26,
 'neutral': 27}

In [42]:
df.iloc[2][7]

C:\Users\ree\AppData\Local\Temp\ipykernel_13280\4098464888.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.iloc[2][7]


[0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [43]:
df.iloc[3][7]

C:\Users\ree\AppData\Local\Temp\ipykernel_13280\67349872.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.iloc[3][7]


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [44]:
df.iloc[7]

text              We need more boards and to create a bit more s...
class                                                          8,20
id                                                          ef4qmod
class list                                                  [8, 20]
class length                                                      2
emotion                                          [desire, optimism]
clean text              need board create bit space name we’ll good
emotion vector    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
Name: 7, dtype: object

In [45]:
df.iloc[7][7]

C:\Users\ree\AppData\Local\Temp\ipykernel_13280\1043638000.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.iloc[7][7]


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [46]:
df = df[df["class length"]!= 5]

In [ ]:
df.to_csv(r"..\dataset\data\readyds\newds.csv", index=False)

In [48]:
df['class length'].value_counts()

class length
1    45446
2     8124
3      655
4       37
Name: count, dtype: int64

In [49]:
df.shape

(54262, 8)

In [50]:
df.isna().sum()

text              0
class             0
id                0
class list        0
class length      0
emotion           0
clean text        0
emotion vector    0
dtype: int64